In [ ]:
import pandas as pd

             Мы будем изучать методы визуализации, используя два набора данных:

1 covid_data — данные о показателях заболеваемости за каждый день в различных регионах 187 стран.

In [ ]:
covid_data = pd.read_csv('data/covid_data.csv')
display(covid_data.head())

,date,province/state,country,confirmed,deaths,recovered
0,01/22/2020,Anhui,China,1.0,0.0,0.0
1,01/22/2020,Beijing,China,14.0,0.0,0.0
2,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,01/22/2020,Fujian,China,1.0,0.0,0.0
4,01/22/2020,Gansu,China,0.0,0.0,0.0


     Мы будем работать со следующими столбцами:

 - date — дата наблюдения;
 - province/state — наименование провинции/штата;
 - country — наименование страны;
 - confirmed — общее число зафиксированных случаев на указанный день;
 - deaths — общее число зафиксированных смертей на указанный день;
 - recovered — общее число выздоровлений на указанный день.

2 country_vaccinations — данные о процессе вакцинирования людей в 221 стране мира.

In [44]:
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]
display(vaccinations_data.head())

,country,date,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,Afghanistan,2021-02-22,0.0,0.0,0.0,NaN,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."


Данная таблица содержит следующие столбцы:

 - country — наименование страны;
 - date — дата наблюдения;
 - total_vaccinations — общее число введённых вакцин в стране на указанный день;
 - people_vaccinated — общее число привитых первым компонентом в стране на указанный день;
 - people_vaccinated_per_hundred — процент привитых первым компонентом в стране на указанный день (рассчитывается как );
 - people_fully_vaccinated — общее число привитых вторым компонентом в стране на указанный день (первый компонент уже был введён им ранее);
 - people_fully_vaccinated_per_hundred — процент привитых вторым компонентом в стране на указанный день (рассчитывается как );
 - daily_vaccination — ежедневная вакцинация (число вакцинированных в указанный день);
 - vaccines — комбинации вакцин, используемые в стране.

_____________________________
Обратите внимание, что признаки confirmed, deaths, recovered, total_vaccination, people_vaccinated, people_fully_vaccinated — это суммарные показатели по стране, то есть с каждым днём они должны расти. Такие признаки называют накопительными.
______________________________
Возникает большое желание объединить таблицы. Для этого необходимо учитывать следующие нюансы:

1 В таблице covid_data необходимо предварительно рассчитать суммарное ежедневное число заболевших во всех провинциях/штатах в каждой стране.

2 В таблицах не совпадает число стран, а иногда и их названия. </br>
При объединении таблиц по столбцу мы определённо теряем данные (в данной задаче потери незначительны). Избежать этого можно ручными преобразованиями данных — искать различия в названиях стран в таблицах и преобразовывать их. Однако это не является темой данного модуля.

3 Таблицы имеют разные периоды наблюдений (вакцины появились позже, чем сам вирус). Объединив данные с типом inner, мы можем потерять большое количество наблюдений в таблице covid_data.

                  ПРЕДОБРАБОТКА ДАННЫХ

Опираясь на замечания выше, выполним небольшую предобработку.</br>
                               В таблице ***covid_data***:</br>
1 Группируем таблицу по дате и названию страны и рассчитываем суммарные показатели по всем регионам. Тем самым переходим от данных по регионам к данным по странам:

In [47]:
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

2 Преобразуем даты в формат datetime с помощью функции pd.to_datetime():

In [48]:
covid_data['date'] = pd.to_datetime(covid_data['date'])

3 Создадим признак больных на данный момент (active). Для этого вычтем из общего числа зафиксированных случаев число смертей и число выздоровевших пациентов:

In [49]:
covid_data['active'] = covid_data['confirmed']-covid_data['deaths']-covid_data['recovered']

4 Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей. Для этого отсортируем данные по названиям стран, а затем по датам. После этого произведём группировку по странам и рассчитаем разницу между «вчера и сегодня» с помощью метода diff():

In [53]:
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed']= covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths']= covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()
display(covid_data.head())

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered
881,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN
920,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
963,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1013,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1067,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0


В таблице vaccinations_data достаточно будет преобразовать столбцы в формат datetime:

In [67]:
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

Задание 3.1</br>
За какой период представлены данные в таблице covid_data? В качестве ответа введите даты в формате datetime (без указания времени).</br>
Введите ответ в следующем формате (без пробелов): ГГГГ-ММ-ДД-ГГГГ-ММ-ДД</br>
Подсказка (1 из 1): Для определения периода можно найти максимум и минимум в столбце date таблицы covid_data.

In [63]:
covid_data['date'].describe()
# я нашла с помощью describe()

C:\Users\Пользователь\AppData\Local\Temp\ipykernel_13656\1195451857.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  covid_data['date'].describe()


count                   86785
unique                    494
top       2021-05-28 00:00:00
freq                      194
first     2020-01-22 00:00:00
last      2021-05-29 00:00:00
Name: date, dtype: object

In [69]:
print(f"Данные о заболеваемости предоставлены от {covid_data['date'].min()} до  {covid_data['date'].max()}")

Данные о заболеваемости предоставлены от 2020-01-22 00:00:00 до  2021-05-29 00:00:00


Задание 3.2</br>
За какой период представлены данные в таблице vaccinations_data? В качестве ответа введите даты в формате datetime без указания времени.</br>
Введите ответ в следующем формате (без пробелов): ГГГГ-ММ-ДД-ГГГГ-ММ-ДД</br>
Подсказка (1 из 1): Для определения интервала даты можно найти максимум и минимум в столбца date таблицы vaccination_data.

In [70]:
vaccinations_data['date'].describe()

C:\Users\Пользователь\AppData\Local\Temp\ipykernel_13656\2727065807.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  vaccinations_data['date'].describe()


count                   42795
unique                    279
top       2021-06-30 00:00:00
freq                      216
first     2020-12-02 00:00:00
last      2021-09-06 00:00:00
Name: date, dtype: object

In [72]:
print(f"Данные о вакцинации за период {vaccinations_data['date'].min()} до {vaccinations_data['date'].max()}")

данные о вакцинации за период 2020-12-02 00:00:00 до 2021-09-06 00:00:00


**2020-01-22-2021-05-29**</br>
**2020-12-02-2021-09-06** </br>
Т.о. видим, что Объединение типа inner нежелательно, так как в таком случае выпадает большое число наблюдений из таблиц. Останутся только наблюдения с 2020-12-02 (нижняя граница диапазона дат таблицы vaccination_data) до 2021-05-29 (верхняя граница диапазона дат таблицы covid_data).

  Задание 3.3</br>
С помощью метода merge() объедините таблицы covid_data и vaccinations_data по столбцам date и country.</br>
Тип объединения выставьте так, чтобы в результирующую таблицу попали только наблюдения за период, вычисленный в задании 3.1. То есть в результирующую таблицу должны попасть все записи из таблицы covid_data и из её пересечения с vaccinations_data, но не более. Результат объединения занесите в переменную covid_df. </br>
Сохраните таблицу covid_df — она понадобится нам в следующем юните. 

*Сколько строк и столбцов в таблице covid_df?* </br>
*Введите ответ в виде двух чисел через дефис (например, 333-33): первое число — количество строк, второе число — количество столбцов.*

In [80]:
covid_df = covid_data.merge(vaccinations_data, on=['date', 'country'], how='left')
print(f"Число строк {covid_df.shape[0]}")
print(f"число столбцов {covid_df.shape[1]}")

Число строк 86785
число столбцов 16


В получившейся в задании 3.3 таблице covid_df создайте признаки death_rate — общий процент смертей среди зафиксированных случаев (летальность) и recover_rate — общий процент случаев выздоровления. Данные характеристики рассчитайте как отношение числа смертей (deaths) и числа выздоровлений (recovered) к числу зафиксированных случаев (confirmed) и умножьте результаты на 100%.

In [86]:
covid_df['death_rate']= covid_df['deaths']/covid_df['confirmed']*100
covid_df['recover_rate'] = covid_df['recovered']/covid_df['confirmed']*100

Задание 3.4</br>
Какова максимальная летальность в США (United States) за весь период? Ответ округлите до второго знака после запятой.

In [91]:
round(covid_df[covid_df['country']=='United States']['death_rate'].max(), 2)

7.19

Задание 3.5</br>
Чему равен средний процент выздоровевших в России (Russia)? Ответ округлите до второго знака после запятой.

In [93]:
round(covid_df[covid_df['country']=='Russia']['recover_rate'].mean(), 2)

: 